In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.metric.metric import MetricCalculator
from mesostat.utils.hdf5_io import DataStorage

from lib.gallerosalas.data_fc_db_raw import DataFCDatabase
import lib.analysis.coactivity as coactivity

%load_ext autoreload
%autoreload 2

In [ ]:
params = {}
# params['root_path_data']  = gui_fpath("Path to data collection",  './')
# params['root_path_data'] = '/media/alyosha/Data/TE_data/yasirdata_raw/'
params['root_path_data'] = '/media/aleksejs/DataHDD/work/data/yasir/yasirdata_raw'

In [ ]:
dataDB = DataFCDatabase(params)

In [ ]:
ds = DataStorage('gallerosalas_result_activity.h5')

In [ ]:
mc = MetricCalculator(serial=True, verbose=False)

In [ ]:
print('mice', dataDB.mice)
print('nSessions', len(dataDB.sessions))
print('datatypes', dataDB.get_data_types())
print('nChannel', dataDB.get_nchannels('mou_5'))

In [ ]:
from mesostat.utils.pandas_helper import pd_query, pd_is_one_row

In [ ]:
areas = sorted(set(dataDB.channelAreasDF['Area']))
areaDict = {a : [] for a in areas}

for iCh, chName in enumerate(dataDB.get_channel_labels()):
    rez = pd_is_one_row(pd_query(dataDB.channelAreasDF, {'LOrig' : chName}))[1]
    areaDict[rez['Area']] += [iCh]

fig, ax = plt.subplots(figsize=(4,4))
dataDB.plot_area_lists(ax, areaDict, haveLegend=True)

# 1. Significance

## 1.1. Correlation plots


## 1.2 PCA exploration

## 1.3. Highly uncorrelated channels

In [ ]:
exclQueryLst = [
    {'datatype' : 'bn_trial', 'intervName' : 'PRE'},  # Baseline normalized
    {'mousename' : 'mou_6', 'intervName' : 'REW'},    # No reward for this mouse
]

In [ ]:
coactivity.corr_plot_session_composite(dataDB, mc, 'corr',
                                       dataTypes=['bn_trial', 'bn_session'],
                                       trialTypes=[None, 'Hit', 'CR'],
                                       exclQueryLst=exclQueryLst,
                                       haveMono=False)

**Drop first PCA and explore result**

In [ ]:
coactivity.corr_plot_session_composite(dataDB, mc, 'corr',
                                       dataTypes=['bn_trial', 'bn_session'],
                                       trialTypes=[None, 'Hit', 'CR'],
                                       exclQueryLst=exclQueryLst,
                                       haveMono=False, nDropPCA=1, haveBrain=True)

# 2. Consistency
## 2.1. PCA consistency over mice
### 2.1.1. Angle-based consistency

Tasks
  * Explained variance by phase/session/mouse/trialType
     * Do not separate phases, its meaningless. Compute PCA for all timesteps, then see proj differences for phases
     * Implement HAC correction

  * Global PCA shifts vs session

Approaches:
  * Eval PCA over all data, select strongest components, plot components as function of cofound
  * Eval PCA for cofounds, compare PCA
  
**Plots**:
* Cosine-squared matrix $C^2_{ij} = (R^{1}_{ik}R^{2}_{jk})^2$, where $R^l$ is the PCA-transform
* Consistency metric $E = e^1_i e^2_j C^2_{ij}$, where $e^l$ are the eigenvalues

**Problem**:
The consistency metric $E$ has all necessary ingredients (angles, eigenvalues), but it is not mathematically clear that it behaves the desired way. Solid theory is required for this metric to be useful.

**Alternative approach**:
Try consistency metric $H(\frac{C^2_{ij}}{N})$. Should be great at measuring the sparsity of basis coupling. The challenge is to include eigenvalue priority into this metric, since spread of weak eigenvalues is not as relevant as spread of strong ones.

In [ ]:
for datatype in ['bn_session', 'bn_trial']:
    coactivity.plot_pca_alignment_bymouse(dataDB, datatype=datatype, trialType=None, intervName='DEL')

### 2.1.2. Eigenvalue-based consistency

* Let $x_1$, $x_2$ be some datasets
* Let $R_1$, $R_2$ be the corresponding PCA-transforms 
* Find total variances
    - $V_1 = \sum_i eig_i(x_1) = tr(cov(x_1)) = \sum_i cov_{ii}(x_1)$
    - $V_2 = \sum_i eig_i(x_2) = tr(cov(x_2)) = \sum_i cov_{ii}(x_2)$
* Find explained variances
    - $e_1 = eval(cov(x_1)) = diag(cov(R_1 x_1))$
    - $e_2 = eval(cov(x_2)) = diag(cov(R_2 x_2))$
* Find explained variances using wrong bases
    - $e_{12} = diag(cov(R_2 x_1))$
    - $e_{21} = diag(cov(R_1 x_2))$
* Find representation errors in explained variance ratios
    - $\epsilon_1 = \frac{\sum_i |e^1_i - e^{12}_i|}{2 V_1}$
    - $\epsilon_2 = \frac{\sum_i |e^2_i - e^{21}_i|}{2 V_2}$



* TODO: iter trialType=[hit, cr, all]
* TODO: iter perf=[naive,expert,all]

In [ ]:
coactivity.plot_corr_consistency_l1(dataDB, intervDict)#, dropChannels=[26])

In [ ]:
coactivity.plot_corr_consistency_l1(dataDB, intervDict, nDropPCA=1)#, dropChannels=[26])

In [ ]:
coactivity.plot_corr_consistency_l1_trialtype(dataDB, intervDict, trialTypes=['Hit', 'CR'], datatype='bn_trial')
coactivity.plot_corr_consistency_l1_trialtype(dataDB, intervDict, trialTypes=['Hit', 'CR'], datatype='bn_session')

In [ ]:
coactivity.plot_corr_consistency_l1_trialtype(dataDB, intervDict, nDropPCA=1, trialTypes=['Hit', 'CR'], datatype='bn_trial')
coactivity.plot_corr_consistency_l1_trialtype(dataDB, intervDict, nDropPCA=1, trialTypes=['Hit', 'CR'], datatype='bn_session')

In [ ]:
intervDictNoPre = intervDict.copy()
del intervDictNoPre['PRE']

In [ ]:
coactivity.plot_corr_consistency_l1_phase(dataDB, intervDictNoPre, datatype='bn_trial')
coactivity.plot_corr_consistency_l1_phase(dataDB, intervDictNoPre, datatype='bn_session')

In [ ]:
coactivity.plot_corr_consistency_l1_phase(dataDB, intervDictNoPre, nDropPCA=1, datatype='bn_trial')
coactivity.plot_corr_consistency_l1_phase(dataDB, intervDictNoPre, nDropPCA=1, datatype='bn_session')

## 2.2. PCA consistency over phases
### 2.2.1 Angle-based consistency

In [ ]:
for datatype in ['bn_session', 'bn_trial']:
    coactivity.plot_pca_alignment_byphase(dataDB, intervNames=['TEX', 'REW'], datatype=datatype, trialType=None)